In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Bulldozer-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Bulldozer-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class ModelEvaluationConfig:
    root_dir : Path
    best_model_path: Path
    save_path : Path
    

In [6]:
from bullprediction.utils.common import read_yaml
from bullprediction.constants import *
from bullprediction.utils.common import create_directories, save_object
from bullprediction.entity import DataTransformationConfig

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_transformation(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            train_path=config.train_path,
            test_path=config.test_path,
            train_data=config.train_data,
            test_data=config.test_data,
            preprocessor=config.preprocessor
        )

        return data_transformation_config


    def get_model_evaluation(self) -> ModelEvaluationConfig:
        
        config = self.config.model_tuner.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
        root_dir=config.root_dir,
        best_model_path = config.best_model_path,
        save_path = config.save_path
         )

        return model_evaluation_config

        

    

In [8]:
from pathlib import Path
from bullprediction.utils.common import save_json  
import numpy as np
from sklearn.metrics import mean_squared_log_error
from bullprediction.conponents.data_transformation import DataTransformation
import joblib  # for loading the model
import os
import json


In [9]:

class ModelEvaluator:
    def __init__(self, config:ModelEvaluationConfig, data_transformer:DataTransformation):
        self.config = config
        self.data_transformer = data_transformer
        self.save_path = config.save_path
        self.model = self._load_model(config.best_model_path)

    def _load_model(self, model_path):
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model file not found at {model_path}")
        return joblib.load(model_path)

    def _rmsle(self, y_true, y_pred):
        """Compute Root Mean Squared Log Error (RMSLE)."""
        y_true = np.clip(y_true, a_min=0, a_max=None)
        y_pred = np.clip(y_pred, a_min=0, a_max=None)
        return np.sqrt(mean_squared_log_error(y_true, y_pred))
    
    def evaluate(self):
        (
            X_train,
            X_val,
            X_test,
            y_train,
            y_val,
            preprocessor_path
        ) = self.data_transformer.initiate_data_transformation_and_split()

        y_val_pred = self.model.predict(X_val)
        val_rmsle = self._rmsle(y_val, y_val_pred)

        results = {"validation_rmsle": val_rmsle}
        print(f"[ModelEvaluator] Validation RMSLE: {val_rmsle:.4f}")

        if self.save_path:
            save_json(Path(self.save_path), results)
            print(f"[ModelEvaluator] Results saved to {self.save_path}")

        return results
    


In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation()
    data_transformer = DataTransformation(config=data_transformation_config)
    model_evaluation_config = config.get_model_evaluation()
    model_evaluation= ModelEvaluator(config=model_evaluation_config, data_transformer=data_transformer)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2025-06-28 15:36:05,940: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-28 15:36:05,940: INFO: common: created directory at: artifacts]
[2025-06-28 15:36:05,940: INFO: common: created directory at: artifacts/data_transformation]
[2025-06-28 15:36:05,940: INFO: common: created directory at: artifacts/model_evaluation]
[2025-06-28 15:36:09,225: INFO: data_transformation: Applying preprocessing pipeline to train and test data.]


c:\Users\ainao\Downloads\Projects\Bulldozer-Prediction\bull\lib\site-packages\sklearn\impute\_base.py:637: UserWarning: Skipping features without any observed values: ['MachineHoursCurrentMeter']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\ainao\Downloads\Projects\Bulldozer-Prediction\bull\lib\site-packages\sklearn\impute\_base.py:574: FutureWarning: Currently, when `keep_empty_feature=False` and `strategy="constant"`, empty features are not dropped. This behaviour will change in version 1.8. Set `keep_empty_feature=True` to preserve this behaviour.
  warnings.warn(
c:\Users\ainao\Downloads\Projects\Bulldozer-Prediction\bull\lib\site-packages\sklearn\impute\_base.py:637: UserWarning: Skipping features without any observed values: ['MachineHoursCurrentMeter']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\ainao\Downloads\Projects\Bulldozer-Prediction\bull\lib\site-p

[ModelEvaluator] Validation RMSLE: 0.6328
[2025-06-28 15:36:09,499: INFO: common: json file saved at: artifacts\model_evaluation\evaluation_results.json]
[ModelEvaluator] Results saved to artifacts/model_evaluation/evaluation_results.json
